In [1]:
import sys
sys.path.append('../code/python')

import findspark
import pyspark

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time

In [2]:
! rm -r metastore_db

In [5]:
findspark.init('/home/snakanda/spark-2.2.0-bin-hadoop2.7')

conf = pyspark.SparkConf()
sc = pyspark.SparkContext(appName="vista")

In [6]:
from vista import Vista

In [7]:
sc

<SparkContext master=local[*] appName=vista>

In [ ]:
def downstream_ml_func(features_df, results_dict, layer_index):
    lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.5)
    train_df, test_df = features_df.randomSplit([0.8, 0.2])
    model = lr.fit(train_df)
    predictions = model.transform(test_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    results_dict[layer_index] = evaluator.evaluate(predictions)
    return results_dict

prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 32, 8, 1, 'alexnet', 3, 0, downstream_ml_func,
                  '~/vista/data/foods/foods.csv',
                  '~/vista/data/foods/images',
                  20129, 130, mem_sys_rsv=3)

print(vista.run())
print("Runtime (min): " + str((time.time()-prev_time)/60.0))

In [4]:
sc.stop()